# 📝 Abstract

* This Notebook shows how to leverage model trained by others to get a decent score.
* With the amount of quality kaggle kernels published by kagglers one can easily use finetuned model to get decent score. 
* This Notebook uses [Ragnar's](https://www.kaggle.com/ragnar123) finetuned model weights as a Feature Extractor, kindly check out his [Training Notebook](https://www.kaggle.com/ragnar123/commonlit-readability-roberta-tf) and [Inference Notebook](https://www.kaggle.com/ragnar123/commonlit-readability-roberta-tf-inference/data) 


### Future Work
* Hyperparameter optimization of Ridge model
* Using other Feature Extractors

### Versions
* Version 3 : CV- 0.37 LB- 0.475
* Version 4 : Setting seed = 123 (same as ragnars seed to avoid data leak) CV - 0.3702 LB- 0.474 

# 🚚 Imports

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import gc

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression, Ridge

import tensorflow as tf 
from tensorflow.keras.layers import Input,LSTM,Bidirectional,Embedding,Dense, Conv1D, Dropout , MaxPool1D , MaxPooling1D, GlobalAveragePooling2D , GlobalAveragePooling1D , GlobalMaxPooling1D , concatenate , Flatten
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model,load_model,save_model , model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping ,LearningRateScheduler
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from transformers import TFBertModel, BertTokenizerFast , BertTokenizer , RobertaTokenizerFast , TFRobertaModel , RobertaConfig , TFAutoModel , AutoTokenizer

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)

# ⚙️ Parameters

In [ ]:
max_len = 250
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 123

MODEL=['bert-base-uncased' , 'roberta-base']

model_name = MODEL[1]

In [ ]:
path=[
    "../input/commonlitreadabilityprize/sample_submission.csv",
    "../input/commonlitreadabilityprize/test.csv",
    "../input/commonlitreadabilityprize/train.csv"
]

df_train = pd.read_csv(path[2])
df_test = pd.read_csv(path[1])
df_ss = pd.read_csv(path[0])

In [ ]:
df_train = df_train.drop(['url_legal','license','standard_error'],axis='columns')
df_test = df_test.drop(['url_legal','license'],axis='columns')

In [ ]:
X= df_train['excerpt']
y=df_train['target'].values

X_test = df_test['excerpt']

# Defining Tokenizer

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")
tokenizer1

# Tokenization

In [ ]:
print('tokenization')
train_embeddings = tokenizer1(X.to_list(), truncation = True , padding = 'max_length' , max_length=max_len)
test_embeddings = tokenizer1(X_test.to_list() , truncation = True , padding = 'max_length' , max_length = max_len)

# 📊 Dataset Preperation

In [ ]:
@tf.function
def map_function(encodings):
    input_ids = encodings['input_ids']
    
    return {'input_word_ids': input_ids}

print("generating train and test")    
train = tf.data.Dataset.from_tensor_slices((train_embeddings))
train = (
            train
            .map(map_function, num_parallel_calls=AUTOTUNE)
            .batch(16)
            .prefetch(AUTOTUNE)
        )


test = tf.data.Dataset.from_tensor_slices((test_embeddings))
test = (
        test
        .map(map_function, num_parallel_calls = AUTOTUNE)
        .batch(16)
        .prefetch(AUTOTUNE)
    )

# 🧠 Modelling

#### Replicating Ragnars model architecture 

In [ ]:
def build_roberta_base_model(max_len=max_len ):
    
    transformer = TFAutoModel.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")
    
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = output)
    
    return model

In [ ]:
ragnar_model = build_roberta_base_model()

In [ ]:
ragnar_model.summary()

# Feature Extraction Model

In [ ]:
def feature_extractor(path):
    print("loading weights")
    ragnar_model.load_weights(path)
    x= ragnar_model.layers[-3].output
    model = Model(inputs = ragnar_model.inputs , outputs = x)
    return model

In [ ]:
def get_preds(model,train,test):
    print("Extracting Features from train data")
    train_features = model.predict( train , verbose =1)
    train_features = train_features.last_hidden_state
    train_features = train_features[: , 0 , :]
    print("Extracting Features from train data")
    test_features = model.predict( test , verbose =1)
    test_features = test_features.last_hidden_state
    test_features = test_features[: , 0 , :]
    
    return np.array(train_features , dtype= np.float16) , np.array(test_features , dtype= np.float16) 

# 🎣 Feature Extraction

In [ ]:
#model weight paths
paths=["../input/commonlit-readability-roberta-base/Roberta_Base_123_1.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_2.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_3.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_4.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_5.h5"
      ]

In [ ]:
#1
extraction_model = feature_extractor(paths[0])
train_embeddings1 , test_embeddings1 = get_preds(extraction_model , train , test)

In [ ]:
#2
extraction_model = feature_extractor(paths[1])
train_embeddings2 , test_embeddings2 = get_preds(extraction_model , train , test)

In [ ]:
#3
extraction_model = feature_extractor(paths[2])
train_embeddings3 , test_embeddings3 = get_preds(extraction_model , train , test)

In [ ]:
#4
extraction_model = feature_extractor(paths[3])
train_embeddings4 , test_embeddings4 = get_preds(extraction_model , train , test)

In [ ]:
#5
extraction_model = feature_extractor(paths[4])
train_embeddings5 , test_embeddings5 = get_preds(extraction_model , train , test)

# 🔄 Kfold Training

#### Using Ridge

In [ ]:
def get_preds(train_embeddings , test_embeddings):
    scores=[]
    kfold = KFold(n_splits=5, shuffle= True , random_state= SEED)
    iteration=1
    preds = np.zeros((test_embeddings.shape[0]))
    for train_idx, test_idx in kfold.split(train_embeddings,y):
        print(f'running iteration {iteration}')
        X_train = train_embeddings[train_idx]
        X_test = train_embeddings[test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]

        regression_model = Ridge()
        
        regression_model.fit(X_train,y_train)
        y_pred = regression_model.predict(X_test)

        score = np.sqrt(mse(y_pred,y_test))
        scores.append(score)
        print(f'Fold {iteration} , rmse score: {score}')
        y_preds = regression_model.predict(test_embeddings)
        y_preds=y_preds.reshape(-1)
        preds+=y_preds  
        iteration += 1

    print(f"the average rmse is {np.mean(scores)}")
    return np.array(preds)/5  

In [ ]:
print("***********predicting***********")
preds1 = get_preds(train_embeddings1,test_embeddings1)
print("***********predicting***********")
preds2 = get_preds(train_embeddings2,test_embeddings2)
print("***********predicting***********")
preds3 = get_preds(train_embeddings3,test_embeddings3)
print("***********predicting***********")
preds4 = get_preds(train_embeddings4,test_embeddings4)
print("***********predicting***********")
preds5 = get_preds(train_embeddings5,test_embeddings5)

# 💯 Submission

In [ ]:
preds=(preds1+preds2+preds3+preds4+preds5)/5
preds = preds.tolist()

In [ ]:
sub=pd.DataFrame({'id':df_ss['id'],'target':preds})
sub.to_csv('submission.csv',index=False)
sub.head()

#### Thanks for viewing, drop your suggestions down in the comments below. 🙂